# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [8]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data"
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [9]:
# TODO: Find way not to hard code this
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_16/*.rec")


In [10]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/standard/2023_06_16/20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

In [11]:
def recursive_dictionary():
    return defaultdict(recursive_dictionary)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [12]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [13]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = recursive_dictionary()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2
Skipping file 20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported


/nancy/projects/reward_competition_extention/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported


In [14]:
session_to_trodes_data

defaultdict(<function __main__.recursive_dictionary()>,
            {'20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2': defaultdict(dict,
                         {'20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged': {'timestampoffset': {},
                           'time': {'timestamps': {'description': 'Timestamps',
                             'byte_order': 'little endian',
                             'original_file': '20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firmware': '3.17',
                             'headstage_firm

- Adding the video timestamps

In [15]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2
Current Video Name: 20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync
Current Video Name: 20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2.videoTimeStamps.cameraHWSync


- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [16]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [17]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,decimation,fields,data,filename,direction,id,display_order,clock rate,session_path
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,time,timestamps,Timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,20,<time uint32><systime int64>,"[[307664, 1686928758076131800], [307665, 16869...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<time uint32>,"[[307664], [307665], [307666], [307667], [3076...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,coordinates,Pad locations in microns,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<time uint32>,"[[307664], [307665], [307666], [307667], [3076...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Dout3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<time uint32><state uint8>,"[[307664, 0]]",20230616_111904_standard_comp_to_training_D4_s...,output,ECU_Dout3,4,NaN,/scratch/back_up/reward_competition_extention/...


In [18]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,decimation,fields,data,filename,direction,id,display_order,clock rate,session_path
20,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<time uint32><state uint8>,"[[307664, 0], [646678, 1], [647879, 0], [64867...",20230616_111904_standard_comp_to_training_D4_s...,input,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...
21,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,coordinates,Pad locations in microns,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
22,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<time uint32>,"[[307664], [307665], [307666], [307667], [3076...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
23,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,307664,NaN,<time uint32>,"[[307664], [307665], [307666], [307667], [3076...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
24,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[309047, 0, 0], [309047, 0, 0], [310433, 0, 0...",20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...


- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [19]:
trodes_metadata_df["data"].iloc[0]

array([(  307664, 1686928758076131800), (  307665, 1686928758076134800),
       (  307666, 1686928758076136500), ...,
       (67788420, 1686932131985293600), (67788421, 1686932131985298400),
       (67788422, 1686932131985303100)],
      dtype=[('time', '<u4'), ('systime', '<i8')])

In [20]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [21]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [22]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,direction,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,time,timestamps,Timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",systime,"[1686928758076131800, 1686928758076134800, 168..."
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766..."
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,coordinates,Pad locations in microns,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766..."
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Dout3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,output,ECU_Dout3,4,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0]


In [23]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,direction,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
20,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,input,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
21,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,coordinates,Pad locations in microns,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
22,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766..."
23,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,analog,timestamps,Analog IO timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766..."
24,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 310433, 311819, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [24]:
trodes_metadata_df["recording"].unique()

array(['20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged',
       '20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2'],
      dtype=object)

## Getting the subject information from the metadata

In [25]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["recording"].apply(lambda x: x.replace("-","_").split("subj")[-1].split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted([i.strip().strip(".") for i in x]))

In [26]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["all_subjects"].apply(lambda x: x[0] if len(x) == 1 else np.nan)

In [27]:
trodes_metadata_df["all_subjects"]

0          [1.4]
1          [1.4]
2          [1.4]
3          [1.4]
4          [1.4]
5          [1.4]
6          [1.4]
7          [1.4]
8          [1.4]
9          [1.4]
10         [1.4]
11         [1.4]
12         [1.2]
13         [1.2]
14         [1.2]
15         [1.2]
16         [1.2]
17         [1.2]
18         [1.2]
19         [1.2]
20         [1.2]
21         [1.2]
22         [1.2]
23         [1.2]
24    [1.2, 1.4]
Name: all_subjects, dtype: object

In [28]:
trodes_metadata_df["current_subject"]

0     1.4
1     1.4
2     1.4
3     1.4
4     1.4
5     1.4
6     1.4
7     1.4
8     1.4
9     1.4
10    1.4
11    1.4
12    1.2
13    1.2
14    1.2
15    1.2
16    1.2
17    1.2
18    1.2
19    1.2
20    1.2
21    1.2
22    1.2
23    1.2
24    NaN
Name: current_subject, dtype: object

In [29]:
ALL_SUBJECTS_COL = "all_subjects"

## Dropping all the rows with unneeded metadata

In [30]:
trodes_metadata_df["metadata_dir"].unique()

array(['time', 'raw', 'analog', 'DIO', 'video_timestamps'], dtype=object)

In [31]:
METADATA_TO_KEEP = ['raw', 'DIO', 'video_timestamps']

In [32]:
trodes_metadata_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(METADATA_TO_KEEP)]

In [33]:
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("out")]
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("coordinates")]


In [34]:
trodes_metadata_df = trodes_metadata_df.reset_index(drop=True)

# Getting the first time stamp of each recording

In [35]:
trodes_raw_df = trodes_metadata_df[(trodes_metadata_df["metadata_dir"] == "raw") & (trodes_metadata_df["metadata_file"] == "timestamps")].copy()


In [36]:
trodes_raw_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4
9,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.2],1.2


In [37]:
trodes_raw_df["first_timestamp"] = trodes_raw_df["first_item_data"].apply(lambda x: x[0])

In [38]:
trodes_raw_df["recording"].iloc[0]

'20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged'

In [44]:
recording_to_first_timestamp = trodes_raw_df.set_index('session_dir')['first_timestamp'].to_dict()

In [47]:
recording_to_first_timestamp

{'20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2': 307664}

#TODO: Make sure that the first timestamp is the same for all the recordings in the session

In [48]:
trodes_metadata_df["first_timestamp"] = trodes_metadata_df["session_dir"].map(recording_to_first_timestamp)

In [49]:
trodes_metadata_df["first_timestamp"]

0     307664
1     307664
2     307664
3     307664
4     307664
5     307664
6     307664
7     307664
8     307664
9     307664
10    307664
Name: first_timestamp, dtype: int64

# Getting the event timestamps

In [50]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4


In [61]:
trodes_metadata_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4
5,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.2],1.2
6,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
7,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
8,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647

In [62]:
# trodes_state_df = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"].copy()
trodes_state_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(["DIO", "video_timestamps"])].copy()

In [63]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4
5,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.2],1.2
6,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
7,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
8,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
10,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 310433, 311819, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 

In [64]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: np.column_stack([np.where(x["last_item_data"] == 1)[0], np.where(x["last_item_data"] == 1)[0]+1]), axis=1)

In [65]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: x["event_indexes"][x["event_indexes"][:, 1] <= x["first_item_data"].shape[0] - 1], axis=1)

In [66]:
trodes_state_df["event_timestamps"] = trodes_state_df.apply(lambda x: x["first_item_data"][x["event_indexes"]], axis=1)

In [67]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [..."
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[646678, 647879], [648676, 693876], [694279, ..."
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567..."
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4,[],[]
5,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.2],1.2,[],[]
6,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [..."
7,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567..."
8,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_comp

## Updating the video timestamps

## Syncing up the video frame data

In [78]:
# Getting the rows that are the metadata for the video timestamps
trodes_video_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"] == "video_timestamps"].copy().reset_index(drop=True)

In [79]:
trodes_video_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 310433, 311819, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",NaN


In [80]:
# Making the video timestamps into an evenly distributed array
trodes_video_df["video_timestamps"] = trodes_video_df["first_item_data"].apply(lambda x: np.linspace(x.min(), x.max(), len(x)).astype(int))

In [81]:
trodes_video_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,video_timestamps
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 310433, 311819, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",NaN,"[309047, 309988, 310930, 311872, 312814, 31375..."


In [83]:
# Creating a dictionary of the video timestamps to add to the LFP dataframe
TRODES_VIDEO_DICT = trodes_video_df.set_index('session_dir')['video_timestamps'].to_dict()

In [91]:
# Adding the video timestamps to the LFP dataframe
trodes_state_df["video_timestamps"] = trodes_state_df["session_dir"].map(TRODES_VIDEO_DICT)

In [92]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_timestamps
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [...","[309047, 309988, 310930, 311872, 312814, 31375..."
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[646678, 647879], [648676, 693876], [694279, ...","[309047, 309988, 310930, 311872, 312814, 31375..."
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567...","[309047, 309988, 310930, 311872, 312814, 31375..."
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4,[],[],"[309047, 309988, 310930, 311872, 312814, 31375..."
5,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.2],1.2,[],[],"[309047, 309988, 310930, 311872, 312814, 31375..."
6,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [...","[309047, 309988, 310930, 311872, 312814, 31375..."
7,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567...","[3

## Finding the closest frame to each event

In [ ]:
trodes_state_df["video_timestamps"].iloc[0]

array([  309047,   309988,   310930, ..., 67786006, 67786948, 67787890])

In [ ]:
trodes_state_df["event_timestamps"].iloc[1][0,0]

646678

In [ ]:
vfunc = np.vectorize(utilities.helper.find_closest_index)

In [ ]:
utilities.helper.find_closest_index(trodes_state_df["video_timestamps"].iloc[1], trodes_state_df["event_timestamps"].iloc[1][0,0])

358

In [ ]:
trodes_state_df[trodes_state_df["event_timestamps"]].apply(lambda x: vfunc(x["video_timestamps"], x["event_timestamps"]), axis=1)

In [107]:
trodes_state_df["event_frame"] = trodes_state_df.apply(lambda x: vfunc(x["video_timestamps"], x["event_timestamps"]), axis=1)

IndexError: invalid index to scalar variable.

In [108]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_timestamps
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [...","[309047, 309988, 310930, 311872, 312814, 31375..."
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[646678, 647879], [648676, 693876], [694279, ...","[309047, 309988, 310930, 311872, 312814, 31375..."
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567...","[309047, 309988, 310930, 311872, 312814, 31375..."
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4,[],[],"[309047, 309988, 310930, 311872, 312814, 31375..."
5,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.2],1.2,[],[],"[309047, 309988, 310930, 311872, 312814, 31375..."
6,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [...","[309047, 309988, 310930, 311872, 312814, 31375..."
7,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567...","[3

## Combine raw and state dataframes

In [ ]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps
1,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [..."
2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 646678, 647879, 648676, 693876, 69427...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[646678, 647879], [648676, 693876], [694279, ..."
3,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567..."
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.4],1.4,[],[]
5,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,[307664],state,[0],[1.2],1.2,[],[]
6,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 32080654, 32081454, 32082454, 3208385...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[32080654, 32081454], [32082454, 32083854], [..."
7,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 395679, 595679, 2795675, 2995676, 509...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[395679, 595679], [2795675, 2995676], [509567..."
8,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_comp

In [ ]:
trodes_raw_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.4],1.4
9,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,raw,timestamps,Raw timestamps,little endian,20230616_111904_standard_comp_to_training_D4_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[307664, 307665, 307666, 307667, 307668, 30766...",time,"[307664, 307665, 307666, 307667, 307668, 30766...",[1.2],1.2


In [ ]:
updated_trodes_metadata_df = pd.concat([trodes_raw_df, trodes_state_df], axis=0)

In [ ]:
updated_trodes_metadata_df.to_pickle(os.path.join(OUTPUT_DIR, OUTPUT_PREFIX + "_trodes_metadata.pkl"))

In [ ]:
raise ValueError("Stop here")

In [ ]:
example_data_arr = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"]["data"].iloc[0]

In [ ]:
example_state_arr = example_data_arr["state"]

In [ ]:
# Assuming 'array' is your numpy array
indices = np.where((example_arr[:-1, 1] == 1) & (example_arr[1:, 1] == 0))[0]
tuple_pairs = array[indices]

In [ ]:
indices = np.where(array == 1)[0]

In [ ]:
indices = np.where((example_arr[:-1] == 1) & (example_arr[1:] == 0))[0] + 1

In [ ]:
indices = np.where((example_arr[:-1] == 1) & (example_arr[1:] == 0))[0]

In [ ]:
indices

In [ ]:
# Assuming 'array' is your numpy array
indices = np.where((example_arr[:-1, 1] == 1) & (example_arr[1:, 1] == 0))[0]
tuple_pairs = array[indices]

In [ ]:
trodes_metadata_df["last_item_data"]

In [ ]:
trodes_metadata_df.to_pickle(os.path.join(OUTPUT_DIR, OUTPUT_PREFIX + "_trodes_metadata.pkl"))